# 3D Beam Buckling Analysis

This notebook tests a vertical beam under compression to compute critical load factors and visualize the buckling mode.

In [ ]:
# Import required modules
import numpy as np
import Direct_Stiffness_Method as dsm
import functions as fu
import Elastic_Critical_Solver as ecs
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Ensure plots display in the notebook
%matplotlib inline

In [ ]:
# Define nodes: Fixed base at (0,0,0), free top at (0,0,5)
nodes = [
    {'node_id': 0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'bc': [True, True, True, True, True, True]},
    {'node_id': 1, 'x': 0.0, 'y': 0.0, 'z': 5.0, 'bc': [False, False, False, False, False, False]}
]

# Material and section properties
A = 0.01      # m²
E = 200e9     # Pa (steel)
nu = 0.3      # Poisson's ratio
Iy = 0.0001   # m⁴
Iz = 0.0001   # m⁴
J = 0.0002    # m⁴

# Create element
print('Creating element...')
element = dsm.create_element(nodes[0], nodes[1], E, nu, A, Iy, Iz, J)
elements = [element]

# Define loads: 1000 N downward at Node 1
loads = {1: np.array([0.0, 0.0, -1000.0, 0.0, 0.0, 0.0])}

In [ ]:
# Inspect the element
print('Element properties:')
print(f"Length: {element['L']}")
print(f"Rotation Matrix:\n{element['rotation_matrix']}")
print(f"Local Stiffness Matrix (first few terms):\n{element['local_stiffness_matrix'][:3, :3]}")
print(f"Global Stiffness Matrix (first few terms):\n{element['global_stiffness_matrix'][:3, :3]}")

In [ ]:
# Run the solver
print('Running CriticalLoadSolver...')
solver = ecs.CriticalLoadSolver(nodes, elements, loads, include_interactions=False)

# Debugging: Check displacements and internal forces
displacements, _ = dsm.calculate_structure_response(nodes, elements, loads)
print('Displacements:', displacements)
internal_forces = solver.compute_internal_forces_and_moments(displacements)
print('Internal Forces for Element (0,1):', internal_forces[(0, 1)])
fx2, mx2, my1, mz1, my2, mz2 = ecs.moments_from_forces(internal_forces, (0, 1))
print('Fx2:', fx2)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = solver.compute_eigenvalues()
print('Critical Load Factors:', eigenvalues[:3])

# Check global matrices
print('Global Elastic Stiffness Matrix (first few terms):\n', dsm.assemble_global_stiffness_matrix(nodes, elements)[:6, :6])
print('Global Geometric Stiffness Matrix (first few terms):\n', solver.global_geo_matrix[:6, :6])

In [ ]:
# Plot the buckling mode
print('Plotting mode shape...')
solver.plot_mode_shape(eigenvectors[:, 0], scale_factor=0.5)